# Билет _

# Задание 2

In [65]:
import numpy as np

In [66]:
A = np.array([[8, 4, 15, 6, 9],
              [3, 16, 13, 8, 10],
              [0, 9, 18, 17, 5],
              [16, 2, 6, 0, 10],
              [18, 13, 9, 17, 8]])

In [67]:
min_x = A.min(axis=1)
min_y = A.min(axis=0)

In [68]:
B = min_x.reshape(5, -1) + min_y
B

array([[ 4,  6, 10,  4,  9],
       [ 3,  5,  9,  3,  8],
       [ 0,  2,  6,  0,  5],
       [ 0,  2,  6,  0,  5],
       [ 8, 10, 14,  8, 13]])

# Задание 3

In [69]:
import math
import numpy as np
import multiprocessing as mp

In [70]:
def foo(ints_sample):
    limit = 0.99999999
    result = []

    range_ = range(len(ints_sample))
    for i in range_:
        left = ints_sample[i]
        for j in range_:
            right = ints_sample[j]
            sin_mul = math.sin(left * right)
            if sin_mul > limit:
                result.append((left, right, sin_mul))

    return result

In [71]:
%%file "task_3.py"
import math


def foo(chunk, ints_sample):
    limit = 0.99999999
    result = []

    for i in chunk:
        for j in ints_sample:
            sin_mul = math.sin(i * j)
            if sin_mul > limit:
                result.append((i, j, sin_mul))

    return result

Overwriting task_3.py


In [72]:
def chunkinize(lst, n: int):
    n = len(lst) // n
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [73]:
def single_benchmark(ints_sample):
    return foo(ints_sample)


def mul_benchmark(ints_sample):
    from task_3 import foo

    n = mp.cpu_count()
    chunks = list(chunkinize(ints_sample, n))
    with mp.Pool(n) as pool:
        res = pool.starmap(foo, zip(chunks, [ints_sample] * n))
        return [i for chunk in res for i in chunk]

In [74]:
ints_sample = np.arange(1, 3000 + 1)

In [75]:
single_result = single_benchmark(ints_sample)
print(len(single_result))

mul_result = mul_benchmark(ints_sample)
print(len(mul_result))

# есть пары: (5, 10) и (10, 5)

410
410


In [79]:
print(single_result[:10])
print(mul_result[:10])

[(23, 2253, 0.999999999696513), (57, 2746, 0.9999999998635031), (62, 2536, 0.9999999970500172), (69, 751, 0.999999999696513), (91, 2855, 0.9999999980220771), (114, 1373, 0.9999999998635031), (115, 2253, 0.9999999924128237), (124, 1268, 0.9999999970500172), (129, 2025, 0.9999999983363994), (133, 2746, 0.99999999925685)]
[(23, 2253, 0.999999999696513), (57, 2746, 0.9999999998635031), (62, 2536, 0.9999999970500172), (69, 751, 0.999999999696513), (91, 2855, 0.9999999980220771), (114, 1373, 0.9999999998635031), (115, 2253, 0.9999999924128237), (124, 1268, 0.9999999970500172), (129, 2025, 0.9999999983363994), (133, 2746, 0.99999999925685)]


In [76]:
%%timeit

single_benchmark(ints_sample)

2.84 s ± 142 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
%%timeit

mul_benchmark(ints_sample)

1.08 s ± 96.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [78]:
# параллельное решение эффективнее по скорости